# Nucleic acid coupling chemistries

This notebook implements all base-linker fragments currently available in Fluordynamics.

In [1]:
import pandas as pd
from biopandas.mol2 import PandasMol2
import fluordynamics as fd
import os

In [2]:
import importlib
importlib.reload(fd.ff)

<module 'fluordynamics.ff' from '/mnt/c/Users/fsteffen/Github/fluordynamics/fluordynamics/ff.py'>

In [3]:
cmd_gui = fd.jupyter.connect2pymol()

In [4]:
dyes = [('sCy3', 'C3W'), ('sCy5', 'C5W'), ('Cy7', 'C7N'), ('Cy5.5', 'C55'), ('Cy7.5','C75'), 
 ('Alexa350', 'A35'), ('Alexa488', 'A48'), ('Alexa532', 'A53'), ('Alexa568', 'A56'), ('Alexa594', 'A59'), ('Alexa647', 'A64'), 
 ('Atto390', 'T39'), ('Atto425', 'T42'), ('Atto465', 'T46'), ('Atto488', 'T48'), ('Atto495', 'T49'), ('Atto514', 'T51'), ('Atto520', 'T52'), ('Atto610', 'T61')]

## Labeling on C5 of deoxythymidine

### Geometry optimization

Create Gaussian input file
```
name=MLE_capped
input_folder='in/'
output_folder='out/'

antechamber -i "$input_folder/$name".mol2 -fi mol2 -o "$output_folder/$name".gin -fo gcrt -gv 1 -ge "$output_folder/$name".gesp -ch "$output_folder/$name"_opt -nc 0
```

Gaussian geometry optimization and ESP calculation
```
nproc=12

sed 's/#HF.*/\#P b3lyp\/6-31G\* Opt/g' < "$output_folder/$name".gin | sed '/iop/d' | sed '/.*gesp/d' | sed "/--Link1--/ a %nproc=$nproc" > "$output_folder/$name"_b3lyp_opt.gin

sed '/^[[:space:]]*[A-Z]/d' < "$output_folder/$name".gin | sed 's/SCF/Geom=check SCF/g'| sed 's/\(\%chk=.*\)opt/\1esp/g' | sed "/--Link1--/ a %nproc=$nproc" > "$output_folder/$name"_hf_esp.gin

g09 < "$output_folder/$name"_b3lyp_opt.gin > "$output_folder/$name"_b3lyp_opt.gout && cp "$output_folder/$name"_opt.chk "$output_folder/$name"_esp.chk
```
> **Note:** Make sure that charge and multiplicity are compatible (check log files in case of segmentation errors) files

```
g09 < "$output_folder/$name"_hf_esp.gin > "$output_folder/$name"_hf_esp.gout
```

### Partial charge fitting with RESP

```
antechamber -i "$input_folder/$name".mol2 -fi mol2 -o "$output_folder/$name".ac -fo ac -pf yes -nc 0

capping_group=MLE_capping_groups.dat

n_atom=`awk '$1 == "GROUP" {print $2}' "$input_folder/$capping_group"`
group_constraint=`awk '$1 == "GROUP" {print $3}' "$input_folder/$capping_group"`

respgen -i "$output_folder/$name".ac -o "$output_folder/$name".respin1 -f resp1 -a "$input_folder/$capping_group"
respgen -i "$output_folder/$name".ac -o "$output_folder/$name".respin2 -f resp2 -a "$input_folder/$capping_group"

# since respgen rounds the group constraint to three decimals replace it with the value from the capping group
for i in `seq $(echo $n_atom | wc -w)`;do
    atom=`echo $n_atom | cut -f$i -d' '`
    group=`echo $group_constraint | cut -f$i -d' '`
    sed -z -i "s/$atom\s*-*\w\.\w*$p/$atom  $group/$i" "$output_folder/$name".respin1
    sed -z -i "s/$atom\s*-*\w\.\w*$p/$atom  $group/$i" "$output_folder/$name".respin2
done

espgen -i "$output_folder/$name"_hf_esp.gout -o "$output_folder/$name"_hf_esp.esp
mv QIN "$output_folder"/

resp -O -i "$output_folder/$name".respin1 -o "$output_folder/$name".respout1 -e "$output_folder/$name"_hf_esp.esp -q  "$output_folder"/QIN -t "$output_folder"/qout_stage1 -p "$output_folder"/punch1 -s "$output_folder"/esout1
resp -O -i "$output_folder/$name".respin2 -o "$output_folder/$name".respout2 -e "$output_folder/$name"_hf_esp.esp -q "$output_folder"/qout_stage1 -t "$output_folder"/qout_stage2 -p "$output_folder"/punch2 -s "$output_folder"/esout2

antechamber -i "$output_folder/$name".ac -fi ac -o "$output_folder/$name"_resp.mol2 -fo mol2 -c rc -cf "$output_folder"/qout_stage2 -pf yes -at amber
```

**Hint:** The above two-stage RESP fitting protocol is wrapped in the following script:
```
resp_fit.sh -n MLE_capped -i 'in/' -o 'out/' -g MLE_capping_groups.dat -c 0
```

### Coupling to base and dye
First couple the deoxythymine base and the MLE linker

In [6]:
cmd_gui.load('../fragments/linkers/MLE/out/MLE_capped_resp.mol2')


In [25]:
names_methylene = ['C7','H01','H02']

#base_resn = ('deoxythymidine', 'DTM')
base_resn = ('oxythymidine', 'RUM')

cmd_gui.reinitialize()
cmd_gui.load('../fragments/bases/out/{}.mol2'.format(base_resn[0]))
cmd_gui.load('../fragments/linkers/MLE/out/MLE_capped_resp.mol2')
cmd_gui.remove('MLE_capped_resp and name {}'.format('+'.join(str(i) for i in names_methylene)))
cmd_gui.remove('{} and (name H71 or name H72)'.format(base_resn[0]))
cmd_gui.fuse('{} and name C7'.format(base_resn[0]), 'MLE_capped_resp and name C8 and resn MLE')
cmd_gui.delete('{}'.format(base_resn[0]))
cmd_gui.alter('all', 'type="ATOM"')
cmd_gui.alter('all', 'elem=""') # PyMOL struggles with atom type definitions in mol2 files, therefore let PyMOL guess the elements itself
cmd_gui.set_name('MLE_capped_resp', base_resn[1])
cmd_gui.set_title('MLE',1,base_resn[1])
cmd_gui.unbond('resn {} and name C8'.format(base_resn[1]), 'resn DT and name C7')
cmd_gui.bond('resn {} and name C8'.format(base_resn[1]), 'resn DT and name C7', 2)

In [26]:
fd.ff.pymol_savemol2('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), base_resn[1], overwrite=True)

In [27]:
fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1)

Check passed!


Now, add the fluorophore

In [38]:
base_resn_list = [('deoxythymidine', 'DTM'),
                  ('oxythymidine', 'RUM')]
for base_resn in base_resn_list:

    for name,dye in dyes:
        fd.ff.couple_dye2baselinker(dye, base_resn[1], 'C99', ['C17', 'O98', 'C16'], ['O98', 'C16', 'C17', 'H95', 'H96', 'H97'])
        cmd_gui.alter('all', 'elem=""')
        fd.ff.save_molecule('../fluorlabel/dyes/{}_{}.pdb'.format(dye, base_resn[1]), '{}_{}'.format(dye, base_resn[1]), 'pdb', overwrite=True)
        #fd.ff.update_dye_library({'filename':'{}_DTM'.format(dye), 'dye':name, 'base':'RU+DT', 'linker':'MLE', 'chemistry':'dT-C5', 'position':'internal'}, '../fluorlabel/dyes/dye_library.json', '../fluorlabel/dyes/dye_library.json', overwrite=True)
        fd.ff.update_dye_library({'filename':'{}_{}'.format(dye, base_resn[1]), 'dye':name, 'base':base_resn[1][0:2], 'linker':'MLE', 'chemistry':'U/dT-C5', 'position':'internal'}, '../fluorlabel/dye_library.json', '../fluorlabel/dye_library.json', overwrite=True)


## Labeling via phosphate at 5'-end

### Geometry optimization

Create Gaussian input file
```
name=POS_capped
input_folder='in/'
output_folder='out/'
```

Here, the net charge is -1
```
antechamber -i "$input_folder/$name".mol2 -fi mol2 -o "$output_folder/$name".gin -fo gcrt -gv 1 -ge "$output_folder/$name".gesp -ch "$output_folder/$name"_opt -nc -1
```

Gaussian geometry optimization and ESP calculation
```
nproc=12

sed 's/#HF.*/\#P b3lyp\/6-31G\* Opt/g' < "$output_folder/$name".gin | sed '/iop/d' | sed '/.*gesp/d' | sed "/--Link1--/ a %nproc=$nproc" > "$output_folder/$name"_b3lyp_opt.gin

sed '/^[[:space:]]*[A-Z]/d' < "$output_folder/$name".gin | sed 's/SCF/Geom=check SCF/g'| sed 's/\(\%chk=.*\)opt/\1esp/g' | sed "/--Link1--/ a %nproc=$nproc" > "$output_folder/$name"_hf_esp.gin

g09 < "$output_folder/$name"_b3lyp_opt.gin > "$output_folder/$name"_b3lyp_opt.gout && cp "$output_folder/$name"_opt.chk "$output_folder/$name"_esp.chk
```
> **Note:** Make sure that charge and multiplicity are compatible (check log files in case of segmentation errors) files

```
g09 < "$output_folder/$name"_hf_esp.gin > "$output_folder/$name"_hf_esp.gout
```

### Partial charge fitting with RESP

Run Antechamber with net charge -1
```
antechamber -i "$input_folder/$name".mol2 -fi mol2 -o "$output_folder/$name".ac -fo ac -pf yes -nc -1

capping_group=POS_capping_groups_5prime_DNA.dat

n_atom=`awk '$1 == "GROUP" {print $2}' "$input_folder/$capping_group"`
group_constraint=`awk '$1 == "GROUP" {print $3}' "$input_folder/$capping_group"`

respgen -i "$output_folder/$name".ac -o "$output_folder/$name".respin1 -f resp1 -a "$input_folder/$capping_group"
respgen -i "$output_folder/$name".ac -o "$output_folder/$name".respin2 -f resp2 -a "$input_folder/$capping_group"

# since respgen rounds the group constraint to three decimals replace it with the value from the capping group
for i in `seq $(echo $n_atom | wc -w)`;do
    atom=`echo $n_atom | cut -f$i -d' '`
    group=`echo $group_constraint | cut -f$i -d' '`
    sed -z -i "s/$atom\s*-*\w\.\w*$p/$atom  $group/$i" "$output_folder/$name".respin1
    sed -z -i "s/$atom\s*-*\w\.\w*$p/$atom  $group/$i" "$output_folder/$name".respin2
done

espgen -i "$output_folder/$name"_hf_esp.gout -o "$output_folder/$name"_hf_esp.esp
mv QIN "$output_folder"/

resp -O -i "$output_folder/$name".respin1 -o "$output_folder/$name".respout1 -e "$output_folder/$name"_hf_esp.esp -q  "$output_folder"/QIN -t "$output_folder"/qout_stage1 -p "$output_folder"/punch1 -s "$output_folder"/esout1
resp -O -i "$output_folder/$name".respin2 -o "$output_folder/$name".respout2 -e "$output_folder/$name"_hf_esp.esp -q "$output_folder"/qout_stage1 -t "$output_folder"/qout_stage2 -p "$output_folder"/punch2 -s "$output_folder"/esout2

antechamber -i "$output_folder/$name".ac -fi ac -o "$output_folder/$name"_resp.mol2 -fo mol2 -c rc -cf "$output_folder"/qout_stage2 -pf yes -at amber
```

**Hint:** The above two-stage RESP fitting protocol is wrapped in the following script:
```
resp_fit.sh -n POS_capped -i 'in/' -o 'out/' -g POS_capping_groups_5prime_DNA.dat -c -1
```

### Coupling of base and dye
Again, first couple the bases and the POS linker

In [532]:
names_methyl = ['C01','H01','H02','H03']
names_phosphate = ['P','O1P','O5\'','O2P']


base_resn = ('deoxythymidine', 'DTP')
# base_resn = ('deoxyadenosine', 'DAP')
# base_resn = ('deoxyguanosine', 'DGP')
# base_resn = ('deoxycytidine', 'DCP')
# base_resn = ('uridine', 'RUP')
# base_resn = ('adenosine', 'RAP')
# base_resn = ('guanosine', 'RGP')
# base_resn = ('cytidine', 'RCP')

cmd_gui.reinitialize()
cmd_gui.load('../fragments/bases/out/{}.mol2'.format(base_resn[0]))

if 'D' in base_resn[1]:
    POS_capped_resp = 'POS_capped_resp_5prime_DNA'
else:
    POS_capped_resp = 'POS_capped_resp_5prime_RNA'
    
cmd_gui.load('../fragments/linkers/POS/out/{}.mol2'.format(POS_capped_resp))
cmd_gui.align('{} and name {}'.format(POS_capped_resp, '+'.join(str(i) for i in names_phosphate)), '{} and (name P or name O1P or name O2P or name O5\')'.format(base_resn[0]))
cmd_gui.remove('{} and name {}'.format(POS_capped_resp, '+'.join(str(i) for i in names_methyl+names_phosphate)))
cmd_gui.create(base_resn[1], '{} or {}'.format(POS_capped_resp, base_resn[0]))
cmd_gui.delete(base_resn[0])
cmd_gui.delete(POS_capped_resp)
cmd_gui.bond('{} and name P'.format(base_resn[1]), '{} and name O01'.format(base_resn[1]))
cmd_gui.alter('all', 'type="ATOM"')
cmd_gui.alter('all', 'elem=""')
cmd_gui.set_title(base_resn[1],1,base_resn[1])

In [533]:
fd.ff.pymol_savemol2('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), base_resn[1], overwrite=True)

In [534]:
if 'D' in base_resn[1]:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1.3079)
else:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1.3081)

Check passed!


Then add the fluorophore

In [39]:
base_resn_list = [('deoxythymidine', 'DTP'), 
                  ('deoxyadenosine', 'DAP'),
                  ('deoxyguanosine', 'DGP'),
                  ('deoxycytidine', 'DCP'),
                  ('uridine', 'RUP'),
                  ('adenosine', 'RAP'),
                  ('guanosine', 'RGP'),
                  ('cytidine', 'RCP')]

for base_resn in base_resn_list:

    for name, dye in dyes:
        fd.ff.couple_dye2baselinker(dye, base_resn[1], 'C99', ['C17', 'O98', 'C16'], ['O98', 'C16', 'C17', 'H95', 'H96', 'H97'])
        cmd_gui.alter('all', 'elem=""')
        fd.ff.save_molecule('../fluorlabel/dyes/{}_{}.pdb'.format(dye, base_resn[1]), '{}_{}'.format(dye, base_resn[1]), 'pdb', overwrite=True)
        for base in ['RA', 'RG', 'RC', 'RU', 'DA', 'DG', 'DC', 'DT']:
            fd.ff.update_dye_library({'filename':'{}_{}P'.format(dye, base), 'dye':name, 'base':base, 'linker':'POS', 'chemistry':'phosphate', 'position':"5'-end"}, '../fluorlabel/dye_library.json', '../fluorlabel/dye_library.json', overwrite=True)

## Labeling via phosphate at 3'-end

Same as for 5'-end. For RESP use instead:
```
resp_fit.sh -n POS_capped -i 'in/' -o 'out/' -g POS_capping_groups_3prime_DNA.dat -c -1
resp_fit.sh -n POS_capped -i 'in/' -o 'out/' -g POS_capping_groups_3prime_RNA.dat -c -1
```

First couple the bases and the POS linker, then add the dye.

In [584]:
names_methyl = ['C01','H01','H02','H03']

base_resn = ('deoxythymidine', 'DTO')
# base_resn = ('deoxyadenosine', 'DAO')
# base_resn = ('deoxyguanosine', 'DGO')
# base_resn = ('deoxycytidine', 'DCO')
# base_resn = ('uridine', 'RUO')
# base_resn = ('adenosine', 'RAO')
# base_resn = ('guanosine', 'RGO')
# base_resn = ('cytidine', 'RCO')

cmd_gui.reinitialize()
cmd_gui.load('../fragments/bases/out/{}.mol2'.format(base_resn[0]))

if 'D' in base_resn[1]:
    POS_capped_resp = 'POS_capped_resp_3prime_DNA'
else:
    POS_capped_resp = 'POS_capped_resp_3prime_RNA'
    
cmd_gui.load('../fragments/linkers/POS/out/{}.mol2'.format(POS_capped_resp))
cmd_gui.alter('{} and name P'.format(POS_capped_resp),'name="P1"')
cmd_gui.alter('{} and name O1P'.format(POS_capped_resp),'name="O3P"')
cmd_gui.alter('{} and name O2P'.format(POS_capped_resp),'name="O4P"')
cmd_gui.pair_fit('{} and name O5\''.format(POS_capped_resp),'{} and name O3\''.format(base_resn[0]), 
                 '{} and name C01'.format(POS_capped_resp), '{} and name C3\''.format(base_resn[0]), 
                 '{} and name H03'.format(POS_capped_resp), '{} and name H3\''.format(base_resn[0]))
cmd_gui.remove('{} and (name {} or name O5\')'.format(POS_capped_resp, '+'.join(str(i) for i in names_methyl)))
cmd_gui.create(base_resn[1], '{} or {}'.format(POS_capped_resp, base_resn[0]))
cmd_gui.delete(base_resn[0])
cmd_gui.delete(POS_capped_resp)
cmd_gui.bond('{} and name P1 and resn POS'.format(base_resn[1]), '{} and name O3\''.format(base_resn[1]))
cmd_gui.alter('all', 'type="ATOM"')
cmd_gui.alter('all', 'elem=""')
cmd_gui.set_title(base_resn[1],1,base_resn[1])

In [585]:
fd.ff.pymol_savemol2('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), base_resn[1], overwrite=True)

In [586]:
if 'D' in base_resn[1]:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1.6921)
else:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1.6919)

Check passed!


In [40]:
base_resn_list = [('deoxythymidine', 'DTO'),
                  ('deoxyadenosine', 'DAO'),
                  ('deoxyguanosine', 'DGO'),
                  ('deoxycytidine', 'DCO'),
                  ('uridine', 'RUO'),
                  ('adenosine', 'RAO'),
                  ('guanosine', 'RGO'),
                  ('cytidine', 'RCO')]

for base_resn in base_resn_list:

    for name, dye in dyes:
        fd.ff.couple_dye2baselinker(dye, base_resn[1], 'C99', ['C17', 'O98', 'C16'], ['O98', 'C16', 'C17', 'H95', 'H96', 'H97'])
        cmd_gui.alter('all', 'elem=""')
        fd.ff.save_molecule('../fluorlabel/dyes/{}_{}.pdb'.format(dye, base_resn[1]), '{}_{}'.format(dye, base_resn[1]), 'pdb', overwrite=True)
        for base in ['RA', 'RG', 'RC', 'RU', 'DA', 'DG', 'DC', 'DT']:
            fd.ff.update_dye_library({'filename':'{}_{}O'.format(dye, base), 'dye':name, 'base':base, 'linker':'POS', 'chemistry':'phosphate', 'position':"3'-end"}, '../fluorlabel/dye_library.json', '../fluorlabel/dye_library.json', overwrite=True)

## Internal labeling via ethenoadenine or ethenocytosine

### Geometry optimization

Same as for deoxythymine (MLE) but with `name=ETH_capped`

### Partial charge fitting with RESP

In general the same as for deoxythymine (MLE). For RESP use:

```
resp_fit.sh -n ETH_capped -i 'in/' -o 'out/' -g ETH_capping_groups_adenine_DNA.dat -c 0
resp_fit.sh -n ETH_capped -i 'in/' -o 'out/' -g ETH_capping_groups_cytosine_DNA.dat -c 0
resp_fit.sh -n ETH_capped -i 'in/' -o 'out/' -g ETH_capping_groups_adenine_RNA.dat -c 0
resp_fit.sh -n ETH_capped -i 'in/' -o 'out/' -g ETH_capping_groups_cytosine_RNA.dat -c 0
```

In [31]:
names_amine = ['N1','N6','H11','H12','H61','H62']

base_resn = ('deoxyadenosine', 'DAE')
#base_resn = ('deoxycytidine', 'DCE')
#base_resn = ('adenosine', 'RAE')
#base_resn = ('cytidine', 'RCE')

cmd_gui.reinitialize()
cmd_gui.load('../fragments/bases/out/{}.mol2'.format(base_resn[0]))

if 'A' in base_resn[1]:
    cmd_gui.load('../fragments/linkers/ETH/out/ETH_capped_resp_{}.mol2'.format(base_resn[0]))
    cmd_gui.pair_fit('ETH_capped_resp and name N1','{} and name N1'.format(base_resn[0]), 
                 'ETH_capped_resp and name N6', '{} and name N6'.format(base_resn[0]),
                 'ETH_capped_resp and name H11', '{} and name C2'.format(base_resn[0]))
else:
    cmd_gui.load('../fragments/linkers/ETH/out/ETH_capped_resp_{}.mol2'.format(base_resn[0]))
    cmd_gui.pair_fit('ETH_capped_resp and name N1','{} and name N3'.format(base_resn[0]), 
                 'ETH_capped_resp and name N6', '{} and name N4'.format(base_resn[0]),
                 'ETH_capped_resp and name H11', '{} and name C2'.format(base_resn[0]))
    
cmd_gui.remove('ETH_capped_resp and name {}'.format('+'.join(str(i) for i in names_amine)))
if 'A' in base_resn[1]:
    cmd_gui.remove('{} and (name H61 or name H62)'.format(base_resn[0]))
    cmd_gui.remove('{} and name H61'.format(base_resn[0]))
    cmd_gui.unbond('{} and name C6'.format(base_resn[0]), '{} and name N6'.format(base_resn[0]))
    cmd_gui.bond('{} and name C6'.format(base_resn[0]), '{} and name N6'.format(base_resn[0]), 2)
    cmd_gui.unbond('{} and name C6'.format(base_resn[0]), '{} and name N1'.format(base_resn[0]))
    cmd_gui.bond('{} and name C6'.format(base_resn[0]), '{} and name N1'.format(base_resn[0]), 1)
else:
    cmd_gui.remove('{} and (name H41 or name H42)'.format(base_resn[0]))
    cmd_gui.unbond('{} and name C4'.format(base_resn[0]), '{} and name N4'.format(base_resn[0]))
    cmd_gui.bond('{} and name C4'.format(base_resn[0]), '{} and name N4'.format(base_resn[0]), 2)
    cmd_gui.unbond('{} and name C4'.format(base_resn[0]), '{} and name N3'.format(base_resn[0]))
    cmd_gui.bond('{} and name C4'.format(base_resn[0]), '{} and name N3'.format(base_resn[0]), 1)
    
cmd_gui.create(base_resn[1], 'ETH_capped or {}'.format(base_resn[0]))
cmd_gui.delete(base_resn[0])
cmd_gui.delete('ETH_capped')

if 'A' in base_resn[1]:
    cmd_gui.bond('{} and name N1'.format(base_resn[1]), '{} and name C9'.format(base_resn[1]))
    cmd_gui.bond('{} and name N6'.format(base_resn[1]), '{} and name C7'.format(base_resn[1]))
else:
    cmd_gui.bond('{} and name N3'.format(base_resn[1]), '{} and name C9'.format(base_resn[1]))
    cmd_gui.bond('{} and name N4'.format(base_resn[1]), '{} and name C7'.format(base_resn[1]))
    
cmd_gui.alter('all', 'type="ATOM"')
cmd_gui.alter('all', 'elem=""')
cmd_gui.set_title(base_resn[1],1,base_resn[1])

In [32]:
fd.ff.pymol_savemol2('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), base_resn[1], overwrite=True)

In [33]:
fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -1)

Check passed!


In [41]:
base_resn_list = [('deoxyadenosine', 'DAE'),
                  ('deoxycytidine', 'DCE'),
                  ('adenosine', 'RAE'),
                  ('cytidine', 'RCE')]

for base_resn in base_resn_list:

    for name, dye in dyes:
        fd.ff.couple_dye2baselinker(dye, base_resn[1], 'C99', ['C17', 'O98', 'C16'], ['O98', 'C16', 'C17', 'H95', 'H96', 'H97'])
        cmd_gui.alter('all', 'elem=""')
        fd.ff.save_molecule('../fluorlabel/dyes/{}_{}.pdb'.format(dye, base_resn[1]), '{}_{}'.format(dye, base_resn[1]), 'pdb', overwrite=True)
        for base in ['RA', 'RC', 'DA', 'DC']:    
            fd.ff.update_dye_library({'filename':'{}_{}E'.format(dye, base), 'dye':name, 'base':base, 'linker':'ETH', 'chemistry':'etheno', 'position':'internal'}, '../fluorlabel/dye_library.json', '../fluorlabel/dye_library.json', overwrite=True)

## Labeling via hydrazide at 3'-end

### Geometry optimization
Same as for phosphate (POS) but with `name=HYD_capped`

### Partial charge fitting with RESP
In general the same as for phosphate (POS). For RESP use:

```
resp_fit.sh -n HYD_capped -i 'in/' -o 'out/' -g HYD_capping_groups_adenine_RNA.dat -c -1
resp_fit.sh -n HYD_capped -i 'in/' -o 'out/' -g HYD_capping_groups_guanine_RNA.dat -c -1
resp_fit.sh -n HYD_capped -i 'in/' -o 'out/' -g HYD_capping_groups_cytosine_RNA.dat -c -1
resp_fit.sh -n HYD_capped -i 'in/' -o 'out/' -g HYD_capping_groups_uracil_RNA.dat -c -1
```

In [616]:
names_methoxyl = ['C01','H01','H02','H03', 'O01']
names_amine = ['N97','H94','H93']

# only RNA residues because labeling needs O2' and O3' to form a dialdehyde
base_resn = ('uridine', 'RUH')
base_resn = ('adenosine', 'RAH')
base_resn = ('guanosine', 'RGH')
base_resn = ('cytidine', 'RCH')

cmd_gui.reinitialize()
cmd_gui.load('../fragments/bases/out/{}.mol2'.format(base_resn[0]))
cmd_gui.load('../fragments/linkers/HYD/out/HYD_capped_resp_{}_RNA.mol2'.format(base_resn[0]))

if ('A' in base_resn[1]) or ('G' in base_resn[1]):
    atm = 'N9'
else:
    atm = 'N1'

cmd_gui.pair_fit('HYD_capped and name O5\'','{} and name O5\''.format(base_resn[0]), 
                 'HYD_capped and name C5\'', '{} and name C5\''.format(base_resn[0]), 
                 'HYD_capped and name P', '{} and name P'.format(base_resn[0]),
                 'HYD_capped and name C4\'', '{} and name C4\''.format(base_resn[0]),
                 'HYD_capped and name O4\'', '{} and name O4\''.format(base_resn[0]),
                 'HYD_capped and name N97', '{} and name {}'.format(base_resn[0], atm))
cmd_gui.remove('HYD_capped_resp and name {}'.format('+'.join(str(i) for i in names_methoxyl)))
cmd_gui.remove('HYD_capped_resp and name {}'.format('+'.join(str(i) for i in names_amine)))
cmd_gui.remove('{} and name {}'.format(base_resn[0], 'C*\'+O*\'+O*P*+H*\'*+P'))
cmd_gui.create(base_resn[1], 'HYD_capped_resp or {}'.format(base_resn[0]))
cmd_gui.delete(base_resn[0])
cmd_gui.delete('HYD_capped_resp')
cmd_gui.bond('{} and name {}'.format(base_resn[1], atm), '{} and name C1\''.format(base_resn[1]))
cmd_gui.set_title(base_resn[1],1,base_resn[1])

In [617]:
fd.ff.pymol_savemol2('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), base_resn[1], overwrite=True)

In [618]:
if 'D' in base_resn[1]:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -0.6921)
else:
    fd.ff.check_charge('../fragments/base_linkers/{}.mol2'.format(base_resn[1]), -0.6919)

Check passed!


In [42]:
base_resn_list = [('uridine', 'RUH'),
                  ('adenosine', 'RAH'),
                  ('guanosine', 'RGH'),
                  ('cytidine', 'RCH')]

for base_resn in base_resn_list:

    for name, dye in dyes:
        fd.ff.couple_dye2baselinker(dye, base_resn[1], 'C99', ['C17', 'O98', 'C16'], ['O98', 'C16', 'C17', 'H95', 'H96', 'H97'])
        cmd_gui.alter('all', 'elem=""')
        fd.ff.save_molecule('../fluorlabel/dyes/{}_{}.pdb'.format(dye, base_resn[1]), '{}_{}'.format(dye, base_resn[1]), 'pdb', overwrite=True)
        for base in ['RA', 'RG', 'RC', 'RU']:    
            fd.ff.update_dye_library({'filename':'{}_{}H'.format(dye, base), 'dye':name, 'base':base, 'linker':'HYD', 'chemistry':'hydrazide', 'position':"3'-end"}, '../fluorlabel/dye_library.json', '../fluorlabel/dye_library.json', overwrite=True)

## Forcefield parameterization

First, read in AMBERDYES force field (Graen et al. *JCTC*, **2014**)

In [45]:
amberdyes_ff = fd.ff.Parameters.read_amberdyes(['../forcefields/amberdyes/ffbonded_amberdyes.itp', '../forcefields/amberdyes/ffnonbonded_amberdyes.itp'])

Run **Acpype** on the internal DTM fragment

```
cd ../fragments/acpype

base_linker=DTM
base=DT

base_linker=RUM
base=RU

linker=MLE
filename=../base_linkers/"$base_linker".mol2
sed "s/${base}/${base_linker}/g" "$filename" | sed "s/${linker}/${base_linker}/g" > "$base_linker"_ff.mol2
acpype -i "$base_linker"_ff.mol2 -o gmx -n -1 -a amber -c user
```

In [46]:
baselinkers_itp = {}
baselinkers_ff = {}

Read the itp files into the `fd.ff.Molecule` class and the forcefield modification parameters into the `fd.ff.Parameters` class.

In [47]:
moleculetypes = ['DTM', 'RUM']

for mol in moleculetypes:
    baselinkers_itp[mol] = fd.ff.Molecule.read_molecule('../fragments/acpype/{}_ff.acpype/{}_ff_GMX.itp'.format(mol,mol), 'FLUOR-DYNAMICS')
    baselinkers_itp[mol].change_type('O3\'', 'OS') # residue is internal not terminal
    for a in ['O98', 'C16', 'C17', 'H95', 'H96', 'H97']:
        baselinkers_itp[mol].remove_atom(a)
        baselinkers_ff[mol] = fd.ff.Parameters.read_frcmod('../fragments/acpype/{}_ff.acpype/{}_ff_AC.frcmod'.format(mol,mol), baselinkers_itp[mol].atomtypes_molecule)
        amberdyes_ff.append(baselinkers_ff[mol])

Define the atoms for the specialbonds.

In [48]:
atoms_amberdyes = {'bondtypes' : [['ng', 'cg']],
                   'angletypes': [['c3g', 'ng', 'cg'],
                                  ['hng', 'ng', 'cg'],
                                  ['ng', 'cg', 'og'],
                                  ['ng', 'cg', 'c3g']],
                   'propertypes' : [['c3g', 'c3g', 'cg', 'ng'],
                                    ['hcg', 'c3g', 'cg', 'ng'],
                                    ['c3g', 'cg', 'ng', 'hng'],
                                    ['og', 'cg', 'ng', 'hng'],
                                    ['c3g', 'cg', 'ng', 'c3g'],
                                    ['og', 'cg', 'ng', 'c3g']]}

atoms_linker = {'bondtypes': [['N', 'cg']],
                'angletypes': [['CT', 'N', 'cg'],
                               ['H', 'N', 'cg'],
                               ['N', 'cg', 'og'],
                               ['N', 'cg', 'c3g']],
                'propertypes': [['c3g', 'c3g', 'cg', 'N'],
                                ['hcg', 'c3g', 'cg', 'N'],
                                ['c3g', 'cg', 'N', 'H'],
                                ['og', 'cg', 'N', 'H'],
                                ['c3g', 'cg', 'N', 'CT'],
                                ['og', 'cg', 'N', 'CT']]}

In [49]:
specialbond_ff = fd.ff.Parameters.read_specialbond(amberdyes_ff, atoms_amberdyes, atoms_linker)
amberdyes_ff.append(specialbond_ff)

### Rational for linker RESP charge derivation at 3'/5'-ends via phosphate

Reference: https://ambermd.org/tutorials/advanced/tutorial1/section1.htm

In the AMBER force fields (Cornell, *JACS*, **1995**) the **DNA** nucleotides the charges at the termini is as follows:
- 5'-end nucleotide: -0.3079
- 3'-end nucleotide: -0.6921

For **RNA** the charges are:
- 5'-end nucleotide: -0.3081
- 3'-end nucleotide: -0.6919

<img src='images/DNAparameters_Cornell1995.png' width=800>

#### Labeled DNA 5'-terminus 
The 5'-end nucleotide will become an internal residue (DNA charge: -1; i.e. the phosphate of the linker will be transferred to this nucleotide). The linker will become the new 5'-terminus (DNA charge: -0.3079). For this purpose, the **methylphosphate cap** ($-PO_2-OCH_3$) of the linker (which will be removed) should be constrained to a charge of -1-(-0.3079)=**-0.6921** during the RESP fit.

#### Labeled DNA 3'-terminus
The 3'-end nucleotide will become an internal residue (DNA charge: -1). The linker will become the new 3'-terminus (DNA charge: -0.36921, i.e. $-PO_2-$ will be retained). For this purpose, the **methoxy cap** ($-O-CH_3$) of the linker (which will be removed) should be constrained to a charge of -1-(-0.6021)=**-0.3079** during the RESP fit.

<img src='images/POS_linker.png' width=800>


### Rational for linker RESP charge derivation at 3'-end via hydrazide

Insertion of a hydrazide with sugar ring closure can only occur in riboses (RNA) because it requires a dialdehyde (O2', O3'). The four RNA bases are parameterized such that the partial charges of the base + C1' + H1' = **0.1160**. The NH2 capping group of the linker (which is a placeholder for the base) + C1' + H1' are therefore constrained to **0.1160**. The charges of C1' and H1' are fixed to those of the respective nucleotide in the Amber force field (e.g. for adenosine: C1'=**0.0394** and H1'=**0.2007**)

The methoxy cap (which will be removed) is constrained to -0.3081 such that the remaining fragment will sum up to **-0.6919**.

<img src='images/etheno_hydrazide_linker.png' width=800>

### Rational for linker RESP charge derivation at internal ethenoadenine / ethenocytosine

The amino capping groups of the linker (which will be removed) are constrained to the partial charge of H61+H62 (adenosine/deoxyadenosine) or H41+H42 (cytidine/deoxycytidine) (e.g. for deoxyadenosine 2\*(-0.4167)= **-0.8334**). Thus for deoxyadenosine the linker will be 0-(-0.8334)=**0.8334** which compensates the missing H61/N62 or H41/N42 in the ethenoadenosine / ethenocytidine.

#### 3'/5'-end phosphates
Acpype requires a residue with integral charge (...,-1,0,1,...). Therefore, we will combine the 3'-end and 5'-end fragments into a dinucleotide (charge: -2) using PyMOL.

In [40]:
end = {'base':'DA', '5':'DAP', '3':'DAO'}
# end = {'base':'DG', '5':'DGP', '3':'DGO'}
# end = {'base':'DC', '5':'DCP', '3':'DCO'}
# end = {'base':'DT', '5':'DTP', '3':'DTO'}
# end = {'base':'RA', '5':'RAP', '3':'RAO'}
# end = {'base':'RG', '5':'RGP', '3':'RGO'}
# end = {'base':'RC', '5':'RCP', '3':'RCO'}
# end = {'base':'RU', '5':'RUP', '3':'RUO'}

# end = {'base':'RA', '5':'RAP', '3':'RAH'}
# end = {'base':'RG', '5':'RGP', '3':'RGH'}
# end = {'base':'RC', '5':'RCP', '3':'RCH'}
# end = {'base':'RU', '5':'RUP', '3':'RUH'}

cmd_gui.reinitialize()
cmd_gui.load('../fragments/base_linkers/{}.mol2'.format(end['5']))
cmd_gui.load('../fragments/base_linkers/{}.mol2'.format(end['3']))

# reassigning residue numbers and segment ids preserves the atom numbering after running through acpype
cmd_gui.alter('resn POS and {}'.format(end['5']), 'resi="1"')
cmd_gui.alter('resn POS and {}'.format(end['5']), 'segi="1"')
cmd_gui.alter('resn {}'.format(end['base']), 'resi="2"')
cmd_gui.alter('resn {}'.format(end['base']), 'segi="2"')
cmd_gui.alter('(resn POS or resn HYD) and {}'.format(end['3']), 'resi="3"')
cmd_gui.alter('(resn POS or resn HYD) and {}'.format(end['3']), 'segi="3"')
cmd_gui.remove('{} and resn {}'.format(end['5'], end['base']))
cmd_gui.create('{}_{}'.format(end['5'],end['3']), '{} or {}'.format(end['5'],end['3']))
cmd_gui.bond('(resn {} or resn HYD) and name P'.format(end['base']), 'resn POS and name O01 and resi 1')
cmd_gui.alter('resn POS', 'resn="{}"'.format(end['3']))
cmd_gui.alter('resn {} or resn HYD'.format(end['base']), 'resn="{}"'.format(end['3']))

64

In [753]:
fd.ff.pymol_savemol2('../fragments/acpype/{}_{}_ff.mol2'.format(end['5'], end['3']), '{}_{}'.format(end['5'], end['3']), overwrite=True)

Run **Acpype** on the dinucleotide fragments.

```
cd ../fragments/acpype/
fusion=DAP_DAO
fusion=DGP_DGO
fusion=DCP_DCO
fusion=DTP_DTO
fusion=RAP_RAO
fusion=RGP_RGO
fusion=RCP_RCO
fusion=RUP_RUO

fusion=RAP_RAH
fusion=RGP_RGH
fusion=RCP_RCH
fusion=RUP_RUH
acpype -i "$fusion"_ff.mol2 -o gmx -n -1 -a amber -c user
```

Read the itp files into the `fd.ff.Molecule` class and the forcefield modification parameters into the `fd.ff.Parameters` class.
> The dinucleotides are differentiated by their `subst_id`.

In [50]:
moleculetypes = [{'base':'DA', '5':'DAP', '3':'DAO'},
       {'base':'DG', '5':'DGP', '3':'DGO'},
       {'base':'DC', '5':'DCP', '3':'DCO'},
       {'base':'DT', '5':'DTP', '3':'DTO'},
       {'base':'RA', '5':'RAP', '3':'RAO'},
       {'base':'RG', '5':'RGP', '3':'RGO'},
       {'base':'RC', '5':'RCP', '3':'RCO'},
       {'base':'RU', '5':'RUP', '3':'RUO'},
       {'base':'RU', '5':'RAP', '3':'RAH'},
       {'base':'RU', '5':'RGP', '3':'RGH'},
       {'base':'RU', '5':'RCP', '3':'RCH'},
       {'base':'RU', '5':'RUP', '3':'RUH'}]

for end in moleculetypes:
    fusion_itp = fd.ff.Molecule.read_molecule('../fragments/acpype/{}_{}_ff.acpype/{}_{}_ff_GMX.itp'.format(end['5'],end['3'],end['5'],end['3']), 'FLUOR-DYNAMICS')
    fusion_itp.change_type('O3\'', 'OS') # residue is internal not terminal
    baselinkers_ff['{}_{}'.format(end['5'],end['3'])] = fd.ff.Parameters.read_frcmod('../fragments/acpype/{}_{}_ff.acpype/{}_{}_ff_AC.frcmod'.format(end['5'],end['3'],end['5'],end['3']), fusion_itp.atomtypes_molecule)
    amberdyes_ff.append(baselinkers_ff['{}_{}'.format(end['5'],end['3'])])
        
    ff_mol2 = PandasMol2().read_mol2('../fragments/acpype/{}_{}_ff.mol2'.format(end['5'],end['3']))
    atoms5 = fusion_itp.atoms[(ff_mol2.df['subst_id']==1) | (ff_mol2.df['subst_id']==2)]
    atoms3 = fusion_itp.atoms[(ff_mol2.df['subst_id']==2) | (ff_mol2.df['subst_id']==3)]
    bonds5 = fusion_itp.bonds[fusion_itp.bonds['i'].isin(atoms5['nr']) & fusion_itp.bonds['j'].isin(atoms5['nr'])]
    bonds3 = fusion_itp.bonds[fusion_itp.bonds['i'].isin(atoms3['nr']) & fusion_itp.bonds['j'].isin(atoms3['nr'])]
    impropers5 = fusion_itp.impropers[fusion_itp.impropers['i'].isin(atoms5['nr']) & fusion_itp.impropers['j'].isin(atoms5['nr']) & fusion_itp.impropers['k'].isin(atoms5['nr']) & fusion_itp.impropers['l'].isin(atoms5['nr'])]
    impropers3 = fusion_itp.impropers[fusion_itp.impropers['i'].isin(atoms3['nr']) & fusion_itp.impropers['j'].isin(atoms3['nr']) & fusion_itp.impropers['k'].isin(atoms3['nr']) & fusion_itp.impropers['l'].isin(atoms3['nr'])]
    
    baselinkers_itp[end['5']] = fd.ff.Molecule(end['5'], atoms5, bonds5, None, None, impropers5)
    baselinkers_itp[end['3']] = fd.ff.Molecule(end['3'], atoms3, bonds3, None, None, impropers3)
    for a in ['O98', 'C16', 'C17', 'H95', 'H96', 'H97']:
        baselinkers_itp[end['5']].remove_atom(a)
        baselinkers_itp[end['3']].remove_atom(a)

#### Ethenoadenine and ethenocytosine

```
cd ../fragments/acpype

base_linker=RAE
base=RA

base_linker=RCE
base=RC

base_linker=DAE
base=DA

base_linker=DCE
base=DC

linker=ETH
filename=../base_linkers/"$base_linker".mol2
sed "s/${base}/${base_linker}/g" "$filename" | sed "s/${linker}/${base_linker}/g" > "$base_linker"_ff.mol2
acpype -i "$base_linker"_ff.mol2 -o gmx -n -1 -a amber -c user
```

In [51]:
moleculetypes = ['RAE', 'RCE', 'DAE', 'DCE']

for mol in moleculetypes:
    baselinkers_itp[mol] = fd.ff.Molecule.read_molecule('../fragments/acpype/{}_ff.acpype/{}_ff_GMX.itp'.format(mol,mol), 'FLUOR-DYNAMICS')
    baselinkers_itp[mol].change_type('O3\'', 'OS') # residue is internal not terminal
    for a in ['O98', 'C16', 'C17', 'H95', 'H96', 'H97']:
        baselinkers_itp[mol].remove_atom(a)
        baselinkers_ff[mol] = fd.ff.Parameters.read_frcmod('../fragments/acpype/{}_ff.acpype/{}_ff_AC.frcmod'.format(mol,mol), baselinkers_itp[mol].atomtypes_molecule)
        amberdyes_ff.append(baselinkers_ff[mol])

Write new **ffnonbonded.itp** and **ffbonded.itp** files of the combined forcefield (ff14sb, amberdyes_ff, baselinkers_ff and specialbond_ff) into a directory fluordyes/.

In [52]:
amberdyes_ff.add2ff('ff14sb', '../forcefields/fluordyes/')

Export a residue topology (**rtp**) file with the new base-linkers.

In [53]:
fd.ff.write_rtp('../forcefields/fluordyes/fluordyes.rtp', [baselinkers_itp[mol] for mol in baselinkers_itp.keys()])

Update the **residuetypes.dat** file.

In [56]:
i = 0
for resi in ['DAO', 'DGO', 'DCO', 'DTO', 'DAP', 'DGP', 'DCP', 'DTP', 'DTM', 'RUM', 'DAE', 'DCE']:
    if i == 0:
        fd.ff.update_residuetypes('{} DNA'.format(resi), '../forcefields/amberdyes/residuetypes_amberdyes.dat', '../forcefields/fluordyes/residuetypes.dat', overwrite=True)
    else:
        fd.ff.update_residuetypes('{} DNA'.format(resi), '../forcefields/fluordyes/residuetypes.dat', '../forcefields/fluordyes/residuetypes.dat', overwrite=True)
    i += 1
    
for resi in ['RAO', 'RGO', 'RCO', 'RUO', 'RAP', 'RGP', 'RCP', 'RUP', 'RAH', 'RGH', 'RCH', 'RUH', 'RAE', 'RCE']:
    fd.ff.update_residuetypes('{} RNA'.format(resi), '../forcefields/fluordyes/residuetypes.dat', '../forcefields/fluordyes/residuetypes.dat', overwrite=True)
    i += 1
    
for name,resi in dyes:
    fd.ff.update_residuetypes('{} RNA'.format(resi), '../forcefields/fluordyes/residuetypes.dat', '../forcefields/fluordyes/residuetypes.dat', overwrite=True)
    i += 1
print('{} new entries in residuetypes.dat'.format(i))

45 new entries in residuetypes.dat


Update the **specbond.dat** file.

In [58]:
i = 0
for name, resi1 in dyes:
    for resi2 in ['DAO', 'DGO', 'DCO', 'DTO', 'DAP', 'DGP', 'DCP', 'DTP', 
                  'RAO', 'RGO', 'RCO', 'RUO', 'RAP', 'RGP', 'RCP', 'RUP', 
                  'DTM', 'RUM', 'RAE', 'RCE', 'DAE', 'DCE',
                  'RAH', 'RGH', 'RCH', 'RUH']:
        if i == 0:
            fd.ff.update_specbond('{} C99 1 {} N99 1 0.132 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/amberdyes/specbond_amberdyes.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        else:
            fd.ff.update_specbond('{} C99 1 {} N99 1 0.132 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        i += 1
        
for resi1 in ['DA', 'DG', 'DC', 'DT']:
    for resi2 in ['DAO', 'DGO', 'DCO', 'DTO']:
        fd.ff.update_specbond(' {} O3\' 1 {} P 1 0.155 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        i += 1
    for resi2 in ['DAP', 'DGP', 'DCP', 'DTP']:
        fd.ff.update_specbond('{} O3\' 1  {} P 1 0.155 {} {}'.format(resi2, resi1, resi2, resi1), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        i += 1
    for resi2 in ['DTM', 'DAE', 'DCE']:
        fd.ff.update_specbond('{} O3\' 1  {} P 1 0.155 {} {}'.format(resi2, resi1, resi2, resi1), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        fd.ff.update_specbond(' {} O3\' 1 {} P 1 0.155 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)    
        i += 1
        
for resi1 in ['RA', 'RG', 'RC', 'RU', 'A', 'G', 'C', 'U']:
    for resi2 in ['RAO', 'RGO', 'RCO', 'RUO', 'RAH', 'RGH', 'RCH', 'RUH']:
        fd.ff.update_specbond(' {} O3\' 1 {} P 1 0.155 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        i += 1
    for resi2 in ['RAP', 'RGP', 'RCP', 'RUP']:
        fd.ff.update_specbond('{} O3\' 1  {} P 1 0.155 {} {}'.format(resi2, resi1, resi2, resi1), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        i += 1
    for resi2 in ['RUM', 'RAE', 'RCE']:
        fd.ff.update_specbond('{} O3\' 1  {} P 1 0.155 {} {}'.format(resi2, resi1, resi2, resi1), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)
        fd.ff.update_specbond(' {} O3\' 1 {} P 1 0.155 {} {}'.format(resi1, resi2, resi1, resi2), '../forcefields/fluordyes/specbond.dat', '../forcefields/fluordyes/specbond.dat', overwrite=True)    
        i += 1
print('{} new entries in specbond.dat'.format(i))

658 new entries in specbond.dat
